In [1]:
import scipy.stats
from sklearn.metrics import mean_squared_error
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
regression_results = pd.read_csv("../results/regr_results_fingerprints_plus_feat.csv", index_col=0)
regression_results

,name_short,adduct,matrix,polarity,regressor,observed_value,prediction
0,2-Oxoglutaric acid,+Cl,9AA,negative,Lin_reg,0.000000,8.921198e+14
1,3-Hydroxymethylglutaric acid,+Cl,9AA,negative,Lin_reg,0.000000,9.770028e+17
2,3-Phosphoglyceric acid,+Cl,9AA,negative,Lin_reg,0.000000,-2.075961e+17
3,"4,5-Dihydroorotic acid",+Cl,9AA,negative,Lin_reg,160.795550,-5.418741e+18
4,4-Hydroxyproline,+Cl,9AA,negative,Lin_reg,46.253326,4.089315e+18
...,...,...,...,...,...,...,...
212935,Urocanic acid,[M]-,pNA,positive,GaussianProcess,0.000000,3.578398e+03
212936,Valine,[M]-,pNA,positive,GaussianProcess,0.000000,3.793842e+03
212937,Xanthine,[M]-,pNA,positive,GaussianProcess,0.000000,3.498633e+03
212938,alpha-tocopherol,[M]-,pNA,positive,GaussianProcess,0.000000,3.863232e+03


In [13]:
# compute Spearman's/Pearson's correlation and mean squared error for each matrix/polarity/regressor
regression_metrics = pd.DataFrame(columns = ['matrix', 'polarity', 'regressor', "Spearman's R", 
                                             'S pval', "Pearson's R", 'P pval', 'RMSE', 'RMSE/std', 'non-zero obs'])
counter = 0
for (matrix, polarity, regressor), rows in regression_results.groupby(['matrix', 'polarity', 'regressor']):
    # remove zero intensity molecules if needed (optional)
    rows = rows[rows['observed_value']!=0]
    
    spearman = scipy.stats.spearmanr(rows.observed_value, rows.prediction)
    pearson = scipy.stats.pearsonr(rows.observed_value, rows.prediction)
    mse = mean_squared_error(rows.observed_value, rows.prediction, squared = False)
    mse_std = mse / rows['observed_value'].std()
    regression_metrics.loc[counter] = [matrix, polarity, regressor, spearman[0], spearman[1], 
                                       pearson[0], pearson[1], mse, mse_std, 
                                       rows[rows['observed_value']!=0].shape[0]]
    counter += 1

In [14]:
# select best regressor for each matrix/polarity combination
best_RMSE = regression_metrics.loc[regression_metrics.groupby(['matrix', 'polarity'])["RMSE/std"].idxmin()]
#best_RMSE

best_spear = regression_metrics.loc[regression_metrics.groupby(['matrix', 'polarity'])["Spearman's R"].idxmax()].sort_values("Spearman's R", ascending=False)
best_spear

,matrix,polarity,regressor,Spearman's R,S pval,Pearson's R,P pval,RMSE,RMSE/std,non-zero obs
5,9AA,negative,RandomForest,0.692942,3.219078e-25,0.530198,1.723145e-13,148592.854824,0.860310,167
158,NOR,positive,RandomForest,0.580814,8.211518e-16,0.227513,3.811961e-03,57574.114521,1.067602,160
14,9AA,positive,RandomForest,0.572140,2.731983e-10,0.256327,8.962025e-03,105883.817809,1.054870,103
149,NOR,negative,RandomForest,0.565692,5.598231e-14,0.104896,2.029631e-01,65607.306746,1.176380,149
61,ClCCA,negative,SVR_poly,0.549635,3.079092e-06,0.300261,1.680353e-02,2359.122166,1.073378,63
140,NEDC,positive,RandomForest,0.542057,1.443889e-07,-0.002423,9.827601e-01,10113.246541,1.102239,82
41,CMBT,negative,RandomForest,0.537216,1.642883e-05,0.237864,7.479523e-02,4338.398374,1.024182,57
72,DAN,negative,DecisionTree,0.527756,1.711288e-12,0.144588,7.265787e-02,24225.584574,1.142047,155
86,DAN,positive,RandomForest,0.526301,2.742956e-16,0.340233,4.646221e-07,25459.089768,1.032479,209
115,DHB,negative,SVR_poly,0.504713,9.768259e-06,0.382905,1.165261e-03,13597.651925,1.122540,69


## Results with intensity normalization
Note: in this case, zero intensities were mapped to negative values

In [30]:
regression_results = pd.read_csv("../results/regr_results_all_feat.csv", index_col=0)
# regression_results = pd.read_csv("../results/regr_results_fingerprints.csv", index_col=0)
# regression_results = pd.read_csv("../results/regr_results_mol_feat.csv", index_col=0)


In [31]:
# compute Spearman's/Pearson's correlation and mean squared error for each matrix/polarity/regressor
regression_metrics = pd.DataFrame(columns = ['matrix', 'polarity', 'regressor', "Spearman's R",
                                             'S pval', "Pearson's R", 'P pval', 'RMSE', 'RMSE/std', 'non-zero obs'])
counter = 0
for (matrix, polarity, regressor), rows in regression_results.groupby(['matrix', 'polarity', 'regressor']):
    # remove zero intensity molecules if needed (optional)
    # rows = rows[rows['observed_value']>-0.4]

    spearman = scipy.stats.spearmanr(rows.observed_value, rows.prediction)
    pearson = scipy.stats.pearsonr(rows.observed_value, rows.prediction)
    mse = mean_squared_error(rows.observed_value, rows.prediction, squared = False)
    mse_std = mse / rows['observed_value'].std()
    regression_metrics.loc[counter] = [matrix, polarity, regressor, spearman[0], spearman[1],
                                       pearson[0], pearson[1], mse, mse_std,
                                       rows[rows['observed_value']!=0].shape[0]]
    counter += 1

In [32]:
# select best regressor for each matrix/polarity combination
best_RMSE = regression_metrics.loc[regression_metrics.groupby(['matrix', 'polarity'])["RMSE/std"].idxmin()]
#best_RMSE

best_spear = regression_metrics.loc[regression_metrics.groupby(['matrix', 'polarity'])["Spearman's R"].idxmax()].sort_values("Spearman's R", ascending=False)
best_spear


,matrix,polarity,regressor,Spearman's R,S pval,Pearson's R,P pval,RMSE,RMSE/std,non-zero obs
122,DHB,positive,RandomForest,0.728445,3.404357e-196,0.775303,6.161330e-238,0.839434,0.632536,1183
68,ClCCA,positive,RandomForest,0.706234,2.454631e-179,0.764343,2.633446e-227,0.842168,0.645082,1183
32,CHCA,positive,RandomForest,0.699307,2.126535e-174,0.746584,2.947654e-211,0.864836,0.665480,1183
176,pNA,positive,RandomForest,0.614160,1.389105e-123,0.696659,1.507422e-172,0.858298,0.717860,1183
104,DHAP,positive,RandomForest,0.613610,2.637873e-123,0.674132,1.365989e-157,0.878668,0.739154,1183
50,CMBT,positive,RandomForest,0.610191,1.376825e-121,0.668256,6.613246e-154,0.885285,0.744477,1183
5,9AA,negative,RandomForest,0.549584,2.544998e-94,0.753676,1.679770e-217,0.651395,0.658668,1183
72,DAN,negative,DecisionTree,0.549046,4.199603e-94,0.687977,1.272965e-166,0.713219,0.745547,1183
86,DAN,positive,RandomForest,0.530821,5.796947e-87,0.628116,7.814783e-131,0.842386,0.779333,1183
151,NOR,negative,SVR_poly,0.508807,6.693023e-79,0.715401,4.253615e-186,0.671168,0.712592,1183
